# Ticket to Ride

This notebook explores the map of the Ticket to Ride Europe edition through a network graph implementation. 

The route has been extracted and stored in a CSV file as rows containing data about each routes between a pair of cities, including the colour, length, whether it's a tunnel, and how many locomotives it needs to complete it.

Using the network graph library in SageMath, we are looking for how to find the highest scoring journeys between a given pair of cities. This is more difficult than just finding the shortest journey, for which several well-known algorithms exist. The highest scoring journey is an optimisation between the highest overall score (using a non-linear scoring function) versus the fewest number of turns it takes to complete (i.e. the fewest number of hops/routes between cities).

## Scoring

Scoring table for route length

In [638]:
def score(length):
  x = [0, 1, 2, 4, 7, 0, 15, 0, 21]
  return x[length]

## Import TTR graph

Import route data from CSV file and create edges. Use the score as the weight.

In [639]:
cities = set()
routes = []
edges = []

import csv
with open('ttr-europe.csv') as f:
  f_csv = csv.reader(f) 
  headers = next(f_csv)
  for row in f_csv:
    city1, city2, colour, length, is_tunnel, locos = row
    cities.add(city1)
    cities.add(city2)
    edges.append((city1, city2, score(int(length))))
    routes.append((city1, city2, int(length), colour, is_tunnel, int(locos)))

In [640]:
TTR = Graph(edges, multiedges = True, weighted =  True)

## Create a sandpit

Also create a smaller network subset for sandpit experimentation.

In [641]:
edge_subset = [('Paris', 'Frankfurt', 4),
               ('Marseille', 'Pamplona', 7),
               ('Pamplona', 'Paris', 7),
               ('Marseille', 'Zürich', 2),
               ('Zürich', "München", 2),
               ('München', 'Frankfurt', 2),
               ('Zürich', 'Paris', 4),
               ('Marseille', 'Paris', 7)]


In [642]:
sandpit = Graph(edge_subset, weighted = True)

In [643]:
sandpit.edges()

[('Frankfurt', 'M\xc3\xbcnchen', 2),
 ('Frankfurt', 'Paris', 4),
 ('Marseille', 'Pamplona', 7),
 ('Marseille', 'Paris', 7),
 ('Marseille', 'Z\xc3\xbcrich', 2),
 ('M\xc3\xbcnchen', 'Z\xc3\xbcrich', 2),
 ('Pamplona', 'Paris', 7),
 ('Paris', 'Z\xc3\xbcrich', 4)]

## Basic graph analysis

In [644]:
(TTR.order(), TTR.size())

(47, 101)

In [645]:
TTR.radius()

5

### Degree

In [646]:
TTR.degree(vertices = cities, labels = True)

{'Amsterdam': 4,
 'Angora': 3,
 'Athina': 4,
 'Barcelona': 3,
 'Berlin': 7,
 'Brest': 3,
 'Brindisi': 3,
 'Bruxelles': 5,
 'Bucuresti': 5,
 'Budapest': 6,
 'Cadiz': 2,
 'Constantinople': 5,
 'Danzig': 3,
 'Dieppe': 5,
 'Edinburgh': 2,
 'Erzurum': 3,
 'Essen': 5,
 'Frankfurt': 8,
 'Kharkov': 3,
 'Kyiv': 6,
 'K\xc3\xb8benhavn': 4,
 'Lisboa': 2,
 'London': 5,
 'Madrid': 5,
 'Marseille': 5,
 'Moskva': 3,
 'M\xc3\xbcnchen': 4,
 'Palermo': 3,
 'Pamplona': 7,
 'Paris': 10,
 'Petrograd': 4,
 'Riga': 3,
 'Roma': 4,
 'Rostov': 3,
 'Sarajevo': 4,
 'Sevastapol': 5,
 'Smolensk': 3,
 'Smyrna': 4,
 'Sochi': 3,
 'Sofia': 4,
 'Stockholm': 3,
 'Venezia': 4,
 'Warszawa': 6,
 'Wien': 6,
 'Wilno': 5,
 'Zagreb': 4,
 'Z\xc3\xbcrich': 4}

There are 11 cases of multiple routes between two cities.

In [647]:
TTR.multiple_edges()

[('Bruxelles', 'Paris', 2),
 ('Bruxelles', 'Paris', 2),
 ('Pamplona', 'Paris', 7),
 ('Pamplona', 'Paris', 7),
 ('Frankfurt', 'Paris', 4),
 ('Frankfurt', 'Paris', 4),
 ('Budapest', 'Wien', 1),
 ('Budapest', 'Wien', 1),
 ('Edinburgh', 'London', 7),
 ('Edinburgh', 'London', 7),
 ('Berlin', 'Warszawa', 7),
 ('Berlin', 'Warszawa', 7),
 ('Berlin', 'Frankfurt', 4),
 ('Berlin', 'Frankfurt', 4),
 ('Essen', 'K\xc3\xb8benhavn', 4),
 ('Essen', 'K\xc3\xb8benhavn', 4),
 ('Madrid', 'Pamplona', 4),
 ('Madrid', 'Pamplona', 4),
 ('Dieppe', 'London', 2),
 ('Dieppe', 'London', 2),
 ('K\xc3\xb8benhavn', 'Stockholm', 4),
 ('K\xc3\xb8benhavn', 'Stockholm', 4)]

In [648]:
float(TTR.average_degree())

4.297872340425532

In [649]:
float(TTR.average_distance())

3.955596669750231

Minimal spanning tree

In [650]:
TTR.min_spanning_tree()[0:10] # etc.

[('Amsterdam', 'Bruxelles', 1),
 ('Amsterdam', 'Frankfurt', 2),
 ('Amsterdam', 'London', 2),
 ('Angora', 'Constantinople', 2),
 ('Angora', 'Erzurum', 4),
 ('Athina', 'Smyrna', 2),
 ('Barcelona', 'Madrid', 2),
 ('Barcelona', 'Pamplona', 2),
 ('Berlin', 'Essen', 2),
 ('Berlin', 'Warszawa', 7)]

## Route analysis

In [651]:
TTR.periphery()

['Rostov', 'Edinburgh', 'Sochi', 'Erzurum', 'Cadiz', 'Lisboa', 'Moskva']

Use the sandpit graph for now.

In [652]:
source = 'Marseille'
dest = 'Frankfurt'

In [653]:
sandpit.shortest_path(source, dest, algorithm='Dijkstra_NetworkX', by_weight=True)

['Marseille', 'Z\xc3\xbcrich', 'M\xc3\xbcnchen', 'Frankfurt']

In [654]:
sandpit.shortest_path_length(source, dest, by_weight=True)

6

This is certainly the shortest weighted path, but that's not very useful in TTR. We need the highest-scoring path, within reasonable limits. The score $\sigma$ is a function of: the number of routes $r$, and the total weighted distance $d$. An example score function that can be minimised is $\sigma = 100*d/n$. We only want to look at paths that are equal to—or at most 1-2 units longer than—the minimal number of routes.

In this case, for example, there are four routes from Marseille to Frankfurt with 3 or fewer hops. The given route has $\sigma = 50$. The best (i.e lowest) scoring path is Marseille → Pamplona → Paris → Frankfurt, with $\sigma = 100*3/(7+7+4) = 16.7$, closely followed by Marseille → Paris → Frankfurt with $\sigma = 18.2$.

So, let's look at calculating the scores.

### Scoring routes between cities

In [655]:
paths = sandpit.all_paths(source, dest)
paths

[['Marseille', 'Paris', 'Frankfurt'],
 ['Marseille', 'Paris', 'Z\xc3\xbcrich', 'M\xc3\xbcnchen', 'Frankfurt'],
 ['Marseille', 'Pamplona', 'Paris', 'Frankfurt'],
 ['Marseille',
  'Pamplona',
  'Paris',
  'Z\xc3\xbcrich',
  'M\xc3\xbcnchen',
  'Frankfurt'],
 ['Marseille', 'Z\xc3\xbcrich', 'Paris', 'Frankfurt'],
 ['Marseille', 'Z\xc3\xbcrich', 'M\xc3\xbcnchen', 'Frankfurt']]

Get the score of each path shorter than three hops. Scoring here is just $d/n$.

In [656]:
# Helper function to return a number or the head of a list
def f(x):
  if (isinstance(x, list)):
    return x[0]
  else:
    return x

In [664]:
# Score path p in graph g
def score_path(g, p):
  n = len(p) - 1
  d = sum([f(g.edge_label(p[e], p[e+1])) for e in range(n)])
  return d

In [665]:
[(x, len(x), score_path(sandpit, x)) for x in paths if len(x) <= 4]

[(['Marseille', 'Paris', 'Frankfurt'], 3, 11),
 (['Marseille', 'Pamplona', 'Paris', 'Frankfurt'], 4, 18),
 (['Marseille', 'Z\xc3\xbcrich', 'Paris', 'Frankfurt'], 4, 10),
 (['Marseille', 'Z\xc3\xbcrich', 'M\xc3\xbcnchen', 'Frankfurt'], 4, 6)]

This is fine for the sandpit but the `all_paths()` call is infeasible for the full graph. We could, for example, do a breadth-first search and limit that search to the minimum distance $d$ or $d+1$. We could also look for some other heuristics, to limit the search space, particularly for widely-separated vertices.

The code below finds all vertices within $2d$ of the start and end vertices.

In [659]:
d = 2
p1 = [p for p in TTR.breadth_first_search(source, distance = d)]
p2 = [p for p in TTR.breadth_first_search(dest, distance = d)]
s = set(p1) & set(p2)
s

{'Brest',
 'Bruxelles',
 'Dieppe',
 'Frankfurt',
 'Marseille',
 'M\xc3\xbcnchen',
 'Pamplona',
 'Paris',
 'Venezia',
 'Z\xc3\xbcrich'}

Create a subgraph of just those vertices, and then find all paths under a given distance.

In [660]:
g1 = TTR.subgraph(vertices = s)
g1.remove_multiple_edges()

In [666]:
p1 = g1.all_paths(source, dest)
[(x, len(x), score_path(g1,x)) for x in p1 if len(x) <= 4]

[(['Marseille', 'Z\xc3\xbcrich', 'Paris', 'Frankfurt'], 4, 10),
 (['Marseille', 'Z\xc3\xbcrich', 'M\xc3\xbcnchen', 'Frankfurt'], 4, 6),
 (['Marseille', 'Paris', 'Bruxelles', 'Frankfurt'], 4, 11),
 (['Marseille', 'Paris', 'Frankfurt'], 3, 11),
 (['Marseille', 'Pamplona', 'Paris', 'Frankfurt'], 4, 18)]

Let's wrap this all in a function...

In [677]:
def candidate_vertices(g, u, v):
  d = floor(g.shortest_path_length(u, v)/2 + 1)
  p1 = [c for c in TTR.breadth_first_search(u, distance = d)]
  p2 = [c for c in TTR.breadth_first_search(v, distance = d)]
  s = set(p1) & set(p2)
  s.add(u)
  s.add(v)
  return s
   
# Score all paths between u and v in graph g up to distance min(u,v)+1
def score_paths(g, u, v):
  gs = g.subgraph(vertices = candidate_vertices(g, u, v))
  gs.remove_multiple_edges()
  paths = gs.all_paths(u, v)
  return [{'path': x, 'routes': len(x), 'points': score_path(gs, x), 'score': float(score_path(gs, x)/len(x))} for x in paths]

Top three paths

In [679]:
x = score_paths(TTR, 'Marseille', 'Frankfurt')
x.sort(key = lambda x: x['score'], reverse = True)
x[0:3]

[{'path': ['Marseille', 'Pamplona', 'Paris', 'Frankfurt'],
  'points': 18,
  'routes': 4,
  'score': 4.5},
 {'path': ['Marseille', 'Pamplona', 'Brest', 'Paris', 'Frankfurt'],
  'points': 22,
  'routes': 5,
  'score': 4.4},
 {'path': ['Marseille',
   'Paris',
   'Pamplona',
   'Brest',
   'Dieppe',
   'Bruxelles',
   'Frankfurt'],
  'points': 27,
  'routes': 7,
  'score': 3.857142857142857}]